In [1]:
import pandas as pd
import redis

In [5]:
full_df = pd.read_csv('./ImdbExample/data.tsv', sep='\t')
print(full_df.size)

/tmp/ipykernel_46414/748300429.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  full_df = pd.read_csv('./ImdbExample/data.tsv', sep='\t')


84215817


In [2]:
movie_df = pd.read_csv('./ImdbExample/data.tsv', sep='\t', nrows=50)
movie_df.head()


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [3]:
r = redis.Redis(host='localhost', port=6379)

In [4]:
movie_df.reset_index()

for index, row in movie_df.iterrows():
    mapping = {
        'title' : str(row['originalTitle']),
        'year'  : str(row['startYear']),
        'genres': str(row['genres'])
    }
    r.hset(name='movie:'+str(row['tconst']), mapping=mapping)
    

The following `redis-cli` command registers a Redis Gear function that is triggered every time a Redis Hash is set with key starting with the prefix `movie:` and writes to a Redis Stream the key of the movie in the hash and its title 
```shell
rg.pyexecute "GB().map(lambda x : {'key' : x['key'], 'title' :  execute('HGET', x['key'], 'title')}).foreach(lambda x : execute('XADD', 'movieStream','*', 'code', x['key'], 'title', x['title'])).register('movie:*')"
```

In [15]:
print(r.xrange('movieStream', '-', '1668104745905-0'))

[(b'1668104745904-0', {b'code': b'movie:tt0000001', b'title': b'Carmencita'}), (b'1668104745905-0', {b'code': b'movie:tt0000002', b'title': b'Le clown et ses chiens'})]
